In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re
import json
import cv2
import albumentations
from fastai.callbacks.hooks import num_features_model
from torch.nn import L1Loss


In [2]:
import numpy as np
import torch
import pandas as pd
import random
import string


In [3]:
j = json.load(open('train-object-detect\\annotations.json'))


In [4]:
SZ = 360


In [5]:
class CustomDataset(Dataset):
    def __init__(self, j, aug=None):
        self.j = j
        if aug is not None: aug = get_aug(aug)
        self.aug = aug
    
    def __getitem__(self, idx):
        item = j2anno(self.j[idx])
        if self.aug: item = self.aug(**item)
        im, bbox = item['image'], np.array(item['bboxes'][0])
        im, bbox = self.normalize_im(im), self.normalize_bbox(bbox)
        
        return im.transpose(2,0,1).astype(np.float32), bbox.astype(np.float32)
    
    def __len__(self):
        return len(self.j)
    
    def normalize_im(self, ary):
        return ((ary / 255 - imagenet_stats[0]) / imagenet_stats[1])
    
    def normalize_bbox(self, bbox):
        return bbox / SZ
    

In [23]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.0).label_from_folder())

In [24]:
src

LabelLists;

Train: LabelList (82417 items)
x: ImageList
Image (3, 500, 375),Image (3, 500, 375),Image (3, 500, 375),Image (3, 500, 375),Image (3, 375, 500)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: train;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: train;

Test: None

In [25]:
tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.1,max_lighting=0.2,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [26]:
data = (src.transform(tfms, size=360, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=32).normalize(imagenet_stats))

In [27]:
data

ImageDataBunch;

Train: LabelList (82417 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: train;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: train;

Test: None

In [28]:
class SnakeDetector(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return x.sigmoid_()

In [29]:
learn = Learner(data, SnakeDetector(arch=models.resnet50), loss_func=L1Loss())

In [30]:
learn.split([learn.model.cnn[:6], learn.model.cnn[6:], learn.model.head])


Learner(data=ImageDataBunch;

Train: LabelList (82417 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: train;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: train;

Test: None, model=SnakeDetector(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): Batch

In [31]:
state_dict = torch.load('models\\snake-detection-model.pth')

In [32]:
learn.model.load_state_dict(state_dict['model'])


In [33]:
learn

Learner(data=ImageDataBunch;

Train: LabelList (82417 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: train;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: train;

Test: None, model=SnakeDetector(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): Batch

In [34]:
import os

In [35]:
import os
if not os.path.exists('temp_test'):
    os.makedirs('temp_test')

In [36]:
src.items

array([WindowsPath('train/class-1059/00446a394fb34b7157a1552ae476d5d7.jpg'),
       WindowsPath('train/class-1059/006db98bdfba11fe93f5a00e5e90fb57.jpg'),
       WindowsPath('train/class-1059/00fc27ca58b5b23ef156d40c83593d0a.jpg'),
       WindowsPath('train/class-1059/00fe3058001840b449b34a857b0ece9d.jpg'), ...,
       WindowsPath('train/class-966/feca3e6503ca751693b8ebd990f4e413.jpg'),
       WindowsPath('train/class-966/ff7a4cb8341ebc14247df674c1f32e08.jpg'),
       WindowsPath('train/class-966/ff87d2bd09abc4819585146776f709d0.jpg'),
       WindowsPath('train/class-966/ffec6051ccf293db31d6e38a97cafa6b.jpg')], dtype=object)

In [41]:
src_new = (ImageList.from_folder(path='round1_test\\round1').split_by_rand_pct(0.0).label_from_folder())

In [48]:
str_name = str(src_new.items[0])

In [49]:
str_name

'round1_test\\round1\\00046ff8b04d6fe7f9c5a9d8dca33561.jpg'

In [50]:
learn.predict(open_image(str_name))

(Category class-1625, tensor(3), tensor([0.2266, 0.0735, 0.7445, 0.9382]))

In [52]:
i=0
for filename in src_new.items:
    im = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (360,360), interpolation = cv2.INTER_AREA)
    im_height, im_width, _ = im.shape
    print(str(filename))
    orig_im = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
    orig_im_height, orig_im_width, _ = orig_im.shape
    to_pred = open_image(filename)
    _,_,bbox=learn.predict(to_pred)
    im_original = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
    im_original = cv2.cvtColor(im_original, cv2.COLOR_BGR2RGB)
    im_original.shape
    im_original_width = im_original.shape[1]
    im_original_height = im_original.shape[0]
    bbox_new = bbox
    bbox_new[0] = bbox_new[0]*im_original_width 
    bbox_new[2]= bbox_new[2]*im_original_width
    bbox_new[1] = bbox_new[1]*im_original_height
    bbox_new[3] = bbox_new[3]*im_original_height
    x_min, y_min, x_max, y_max = map(int, bbox_new)
    #cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    im_original = im_original[y_min:y_max,x_min:x_max]
    im_original = cv2.cvtColor(im_original,cv2.COLOR_BGR2RGB)
    filename_str = str(filename)
    to_save = filename_str.replace('round1_test\\round1','temp_test')
    print(to_save)
    cv2.imwrite(to_save,im_original)
    if i==10:
        break
    i+=1


round1_test\round1\00046ff8b04d6fe7f9c5a9d8dca33561.jpg
temp_test\00046ff8b04d6fe7f9c5a9d8dca33561.jpg
round1_test\round1\0004913b37b5293e5a33eb593f8b5aea.jpg
temp_test\0004913b37b5293e5a33eb593f8b5aea.jpg
round1_test\round1\000a06bf72e2d6757b6aabc64b3d6413.jpg
temp_test\000a06bf72e2d6757b6aabc64b3d6413.jpg
round1_test\round1\000a282c4aa89ddee660214d272df598.jpg
temp_test\000a282c4aa89ddee660214d272df598.jpg
round1_test\round1\000a6431024d41527975019019ef58d6.jpg
temp_test\000a6431024d41527975019019ef58d6.jpg
round1_test\round1\001764a8c8128e2d9bab29da705b42c0.jpg
temp_test\001764a8c8128e2d9bab29da705b42c0.jpg
round1_test\round1\001866a40f60e3aa17847325e6a68453.jpg
temp_test\001866a40f60e3aa17847325e6a68453.jpg
round1_test\round1\0020ae3b739cac9ee80336830ebcc19c.jpg
temp_test\0020ae3b739cac9ee80336830ebcc19c.jpg
round1_test\round1\00216fe4c77bceec51552a88a6d2f3fa.jpg
temp_test\00216fe4c77bceec51552a88a6d2f3fa.jpg
round1_test\round1\0021ab7aeea24fd46120d872f1d90f53.jpg
temp_test\0021ab7